# Audio Recognition
### Tim Nguyen & Neeraj Shah
---


In the following code portions below, we implement a barebones version of how audio recognition works using the core concept sof digital signal processing.  This involves sampling the analog signal (sound input), converting it a digital signal, performing a discrete fourier transform, and matching the audio fingerprints with a predefined database.

### Dependencies

1) Need to have PyAudio library.  Install with python's package manager `pip` :

`pip install pyaudio`



In [12]:
# Import required libraries

from sys import byteorder
from array import array
import copy
import pyaudio
import wave
import numpy as np
import matplotlib.pyplot as plt


In [13]:
# Sample the songs in the library to create the TRUE database.



## Code below deals with audio input

`sound` : contains the hex values of what we record. 

In [25]:
# Recording with microphone

chunk = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "output.wav"

p = pyaudio.PyAudio()

stream = p.open(format = FORMAT,
                channels = CHANNELS,
                rate = RATE,
                input = True,
                frames_per_buffer = chunk)

print "* recording"
sound = []
for i in range(0, RATE / chunk * RECORD_SECONDS):
    data = stream.read(chunk, exception_on_overflow = False)
    sound.append(data)
print "* done recording"

stream.close()
p.terminate()

* recording
* done recording


In [22]:
def main():